# Frequentist vs Bayesian Approaches to counting photons an the Metropolis-Hastings algorithm
Here we will look into the issue of frequentist vs baysian probabilities. We will solve the problem of counting photons from the two perspectivs. We will also introduce the  Metropolis-Hastings algorithm, and will show how to use it to infer the parameters of a model, by sampling from a posterior distribution.

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

We first generate some fake but realistic dataset for the fluxes

In [ ]:
# Generating some simple photon count data
import numpy as np
from scipy import stats
np.random.seed(1)  # for repeatability

F_true = 1000  # true flux, say number of photons measured in 1 second
N = 50 # number of measurements
F = stats.poisson(F_true).rvs(N)  # N measurements of the flux
e = np.sqrt(F)  # errors on Poisson counts estimated via square root

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(F, np.arange(N), xerr=e, fmt='ok', ecolor='gray', alpha=0.5)
ax.vlines([F_true], 0, N, linewidth=5, alpha=0.2)
ax.set_xlabel("Flux");ax.set_ylabel("measurement number");

## Frequentist View

If we assume Gaussian errors, then the probability distribution for a single measurement is a normal distribution:

$$P(D_i~|~F_{\rm true}) = \frac{1}{\sqrt{2\pi e_i^2}} \exp{\left[\frac{-(F_i - F_{\rm true})^2}{2 e_i^2}\right]}$$

And we construct the likelihood as the product of all single measurements ($D$):

$$\mathcal{L}(D~|~F_{\rm true}) = \prod_{i=1}^N P(D_i~|~F_{\rm true})$$

It is easier in general to calculate the logarithm of this expression, since the product becomes a sum and we do not loose generality:

$$\log\mathcal{L} = -\frac{1}{2} \sum_{i=1}^N \left[ \log(2\pi  e_i^2) + \frac{(F_i - F_{\rm true})^2}{e_i^2} \right]$$

In this case we can compute the value of $F_true$ that maximizes the likelihood analytically, by setting the derivative of the previous expression to zero, and solving for the parameter, which results in:

$$F_{\rm est} = \frac{\sum w_i F_i}{\sum w_i};~~w_i = 1/e_i^2$$

If all erros are equal, this reduces to the mean, as expected:

$$F_{\rm est} = \frac{1}{N}\sum_{i=1}^N F_i$$

We can evaluate this for our little toy model

In [ ]:
w = 1. / e ** 2
print("""
      F_true = {0}
      F_est  = {1:.0f} +/- {2:.0f} (based on {3} measurements)
      """.format(F_true, (w * F).sum() / w.sum(), w.sum() ** -0.5, N))

## Bayesian View
In this case, we perform the inference directly on the model parameters, which are assumed to be random variables. We infer the physics, not the data. Therefore, we want to estimate:

$$P(F_{\rm true}~|~D)$$

In this case, it makes sense to talk about the probability of the parameters like $F_{\rm{true}}$. We can use the Bayes' rule now to express this posterior distribution as a function of the likelihood and the *prior*:

$$P(F_{\rm true}~|~D) = \frac{P(D~|~F_{\rm true})~P(F_{\rm true})}{P(D)}$$

The prior contains our degree of belief on the parameter having certain value *before* we perform the measurements. This previous knowledge can come from previous similar experiments, or independent theories or measurements that inform us about the value that the parameter *should* have. The new data has a very specific task, then: update our prior believe into a posterior that takes into account the new data. Here we assume that the set of parameters is given by $\theta$, which in this case consists only of the flux, i.e., $\theta=\left[F_{\rm true}\right]$. We start with a flat prior:

In [ ]:
def log_prior(theta):
    return 1  # flat prior

# Define the likelihood
def log_likelihood(theta, F, e):
    return -0.5 * np.sum(np.log(2 * np.pi * e ** 2)
                         + (F - theta[0]) ** 2 / e ** 2)

# In log space, the posterior is the sum of likelihood and prior.
def log_posterior(theta, F, e):
    return log_prior(theta) + log_likelihood(theta, F, e)

We now use emcee to sample this posterior.

In [ ]:
ndim = 1  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nburn = 1000  # "burn-in" period to let chains stabilize
nsteps = 2000  # number of MCMC steps to take

# we'll start at random locations between 0 and 2000
starting_guesses = 2000 * np.random.rand(nwalkers, ndim)

import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[F, e])
sampler.run_mcmc(starting_guesses, nsteps)

sample = sampler.chain  # shape = (nwalkers, nsteps, ndim)
sample = sampler.chain[:, nburn:, :].ravel()  # discard burn-in points

In [ ]:
# plot a histogram of the sample
plt.hist(sample, bins=50, histtype="stepfilled", alpha=0.3, normed=True)

# plot a best-fit Gaussian
F_fit = np.linspace(975, 1025)
pdf = stats.norm(np.mean(sample), np.std(sample)).pdf(F_fit)

plt.plot(F_fit, pdf, '-k')
plt.xlabel("F"); plt.ylabel("P(F)")

## A bit more complicated

In [ ]:
np.random.seed(42)  # for reproducibility
N = 100  # we'll use more samples for the more complicated model
mu_true, sigma_true = 1000, 15  # stochastic flux model

F_true = stats.norm(mu_true, sigma_true).rvs(N)  # (unknown) true flux
F = stats.poisson(F_true).rvs()  # observed flux: true flux plus Poisson errors.
e = np.sqrt(F)  # root-N error, as above

In [ ]:
def log_likelihood(theta, F, e):
    return -0.5 * np.sum(np.log(2 * np.pi * (theta[1] ** 2 + e ** 2))
                         + (F - theta[0]) ** 2 / (theta[1] ** 2 + e ** 2))

# maximize likelihood <--> minimize negative likelihood
def neg_log_likelihood(theta, F, e):
    return -log_likelihood(theta, F, e)

from scipy import optimize
theta_guess = [900, 5]
theta_est = optimize.fmin(neg_log_likelihood, theta_guess, args=(F, e))
print("""
      Maximum likelihood estimate for {0} data points:
          mu={theta[0]:.0f}, sigma={theta[1]:.0f}
      """.format(N, theta=theta_est))

In [ ]:
def log_prior(theta):
    # sigma needs to be positive.
    if theta[1] <= 0:
        return -np.inf
    else:
        return 0

def log_posterior(theta, F, e):
    return log_prior(theta) + log_likelihood(theta, F, e)

# same setup as above:
ndim, nwalkers = 2, 50
nsteps, nburn = 2000, 1000

starting_guesses = np.random.rand(nwalkers, ndim)
starting_guesses[:, 0] *= 2000  # start mu between 0 and 2000
starting_guesses[:, 1] *= 20    # start sigma between 0 and 20

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[F, e])
sampler.run_mcmc(starting_guesses, nsteps)

sample = sampler.chain  # shape = (nwalkers, nsteps, ndim)
sample = sampler.chain[:, nburn:, :].reshape(-1, 2)

In [ ]:
from astroML.plotting import plot_mcmc
fig = plt.figure()
ax = plot_mcmc(sample.T, fig=fig, labels=[r'$\mu$', r'$\sigma$'], colors='k')
ax[0].plot(sample[:, 0], sample[:, 1], ',k', alpha=0.1)
ax[0].plot([mu_true], [sigma_true], 'o', color='red', ms=10);

## Another example showing what happens behind the scene in Mentropolis-Hastings
This is taken from Ridlo W. Wibowo's blog. (http://nbviewer.jupyter.org/github/ridlo/stats_notebook/blob/master/mcmc_metropolis-hastings_pdfsampling.ipynb)

In [ ]:
import math
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
def pdftarget(x, norm=1):
    return np.exp(0.4*(x-0.4)*(x-0.4) - 0.08*x*x*x*x)/norm

In [ ]:
def sampling_mcmc_mh(xt, stepsize, nsamp):
    samples = np.empty(nsamp)
    accept = np.empty(nsamp)
    for i in range(nsamp):
        xprime = xt + stepsize*np.random.normal() # gaussian proposal distribution
        a = pdftarget(xprime)/pdftarget(xt) # symmetric -> gaussian
        if a >= 1.0:
            xt = xprime
            accept[i] = 1
        else:
            u = np.random.random()
            if a >= u:
                xt = xprime
                accept[i] = 1
            else:
                accept[i] = 0 # reject xprime, xt = xt
        
        samples[i] = xt
        
    return samples, accept

In [ ]:
mu = 0
sigma = 2
nsamp = 10000

samples, accept = sampling_mcmc_mh(mu, sigma, nsamp)

In [ ]:
nbins = 50
xmin, xmax = -5, 5

I = quad(pdftarget, -100, +100)
x = np.linspace(xmin, xmax, 1000)
y = pdftarget(x, I[0])

plt.hist(samples, bins=nbins, normed=True, histtype="stepfilled", color="blue", alpha=0.5, linewidth=0)
plt.plot(x, y, 'k')
plt.xlim([xmin, xmax])
plt.show()

### Let's make an animation

In [ ]:
def plot_samples(samples, accept, stepsize, x, target_dist, xmin=-5, xmax=5, nbins=50, write=False, filename="plot_samp_mcmc.png", trace=False):
    nsamples = len(samples)
    ofile = '/home/ridlo/project/stats/mcmc_sampling/'+filename 
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim(xmin, xmax)
    
    #x = np.linspace(xmin, xmax, 1000) 
    #target_dist = pdftarget(x, normed=normalize)
    max_propdist = norm.pdf(0, 0, stepsize) # to draw line
    
    ymax = 1.1*np.amax(target_dist)
    ax.set_ylim(0, ymax)
    
    ax.plot(x, target_dist, 'k') # draw target dist line
    if nsamples > 1:
        ax.hist(samples, normed=True, bins=nbins, histtype="stepfilled", color="blue", alpha=0.5, linewidth=0)
        if trace:
            last_state = samples[-1] # last sample 
            last_acc = accept[-1]
            prev_state = samples[-2]
            prev_acc = accept[-2]
            
            ax.plot(x, norm.pdf(x, prev_state, stepsize), 'g') # draw previous propos dist
            ax.axvline(x=prev_state, ymin=0, ymax=(max_propdist)/(ymax), c='g')
            
            color = 'r'
            if last_acc > 0: color = 'k'     
            ax.axvline(x=last_state, ymin=0, ymax=(norm.pdf(last_state, prev_state, stepsize))/(ymax), c=color)

        ratio_accept = float(len(samples[accept>0]))/float(nsamples)
        text = r'$n_{sample} = '+'{0:d}$'.format(nsamples)+'\n'
        text += r'$r_{accept} = '+'{0:0.2f}$'.format(ratio_accept)
        ax.annotate(text, xy=(0.7, 0.97), xycoords='axes fraction', ha='left', va='top') 
    
    if write:
        plt.savefig(ofile, bbox_inches='tight', dpi=400); plt.close()
    else:
        plt.show(); plt.close()

In [ ]:
# Test
mu = 0
sigma = 2
nsamp = 1000
samples, accept = sampling_mcmc_mh(mu, sigma, nsamp)

I = quad(pdftarget, -100, +100)
x = np.linspace(xmin, xmax, 1000)
y = pdftarget(x, I[0])

In [ ]:
plot_samples(samples, accept, sigma, x, y, trace=True)

In [ ]:
from IPython.display import YouTubeVideo

In [ ]:
YouTubeVideo("zL2lg_Nfi80")